In [1]:
# hera stuff
import hera_cal as hc
import pyuvdata
import aipy
import uvtools

# general
import numpy as np
import random
import matplotlib.pyplot as plt
import sys, io # flushing notebook output

# NN
import tensorflow as tf
from sklearn.model_selection import train_test_split

***

In [2]:
# load in data for a JD. 
data_directory = '../zen_data/'
miriad_path = data_directory + 'zen.2458098.58037.xx.HH.uv'

uvd = pyuvdata.UVData()
uvd.read_miriad(miriad_path)

# gains for same data 
calfits_path = data_directory + 'zen.2458098.58037.xx.HH.uv.abs.calfits'
gains, flags = hc.io.load_cal(calfits_path)

# get the redundancies for that data
aa = hc.utils.get_aa_from_uv(uvd)
info = hc.omni.aa_to_info(aa)
red_bls = np.array(info.get_reds())

***

In [ ]:
# sometimes the antennas are bad, make sure they are good
def ants_good(bl, gain_keys):
    """Returns True if both antennas are in gains.keys"""
    ants = [a[0] for a in gain_keys] # gain_keys is a list of tuples like (ant, pol)
    if bl[0] in ants and bl[1] in ants:
        return True
    else:
        return False
    
def gen_flatness_from_good_red_bls(good_red_bls):

    num_keys = 0
    while num_keys < 2:

        data = {bl: uvd.get_data(bl) for bl in np.random.choice(good_red_bls) if ants_good(bl, gains.keys())}
        num_keys = len(data.keys())
        
    rnd_keys = random.sample(data.keys(), 2)
    
    a, b = rnd_keys[0][0], rnd_keys[0][1]
    c, d = rnd_keys[1][0], rnd_keys[1][1]
    S = data[a, b]*data[c, d].conjugate() * gains[(a,'x')].conjugate() * gains[(b,'x')] * gains[(c,'x')] * gains[(d,'x')].conjugate()
 
    return S

def gen_mega_flatness(good_red_bls, num = 250):
    mega_S = []
    for _ in range(num):
        mega_S.append(gen_flatness_from_good_red_bls(good_red_bls))
    return np.concatenate(mega_S, axis = 0)

def gen_plot(predicted_values, actual_values, output_scaler):
    """Create a prediction plot and save to byte string."""
    
    prediction_unscaled = output_scaler.inverse_transform(predicted_values)
    actual_unscaled = output_scaler.inverse_transform(actual_values)

    sorting_idx = np.argsort(actual_unscaled.T[0])
        
    fig, ax = plt.subplots(figsize = (5, 3), dpi = 144)

    ax.plot(prediction_unscaled.T[0][sorting_idx],
            linestyle = 'none', marker = '.', markersize = 1,
            color = 'darkblue')
    
    ax.plot(actual_unscaled.T[0][sorting_idx],
            linestyle = 'none', marker = '.', markersize = 1, alpha = 0.50,
            color = '#E50000')       
    
    ax.set_title('std: %.9f' %np.std(prediction_unscaled.T[0][sorting_idx] - actual_unscaled.T[0][sorting_idx]))
    
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi = 144)
    plt.close(fig)
    buf.seek(0)

    return buf.getvalue()

def get_acc(predicted_values, actual_values, output_scaler, val = 0.0005):
    prediction_unscaled = output_scaler.inverse_transform(predicted_values)
    actual_unscaled = output_scaler.inverse_transform(actual_values)
    perc = abs(actual_unscaled.T[0] - prediction_unscaled.T[0])
    acc = np.array(perc < val, dtype = int)
    return np.mean(acc)

def clean_phase(num_entries = 15000, tau_low = -0.04, tau_high = 0.04):

    # random.uniform is half open [low, high) not sure if that matters
    tau = np.random.uniform(low = tau_low, high = tau_high, size = num_entries) 
    nu = np.arange(1024)
    
    clean_phase_data = np.empty((num_entries, 1024))
    for i in range(num_entries):
        clean_phase_data[i] = np.angle(np.exp(2*np.pi*1j * (tau[i]*nu + np.random.uniform())))
    
    return clean_phase_data, tau.reshape(-1, 1)

***